In [1]:
import pandas as pd
import json

submissions1 = pd.read_csv("../data/submission.csv")
comments1 = pd.read_csv("../data/comment.csv",index_col=False)

submissions2 = pd.read_csv("../data/submission_rows.csv")
comments2 = pd.read_csv("../data/comment_rows.csv",index_col=False)

In [2]:
print("Number of rows in submissions1:", submissions1.shape[0])
print("Number of rows in submissions2:", submissions2.shape[0])
print("Number of rows in comments1:", comments1.shape[0])
print("Number of rows in comments2:", comments2.shape[0])

Number of rows in submissions1: 332
Number of rows in submissions2: 211
Number of rows in comments1: 53983
Number of rows in comments2: 55573


In [3]:
submissions = pd.concat([submissions1, submissions2]).drop_duplicates(["submission_id","redditor_id"]).reset_index(drop=True)
print("Number of rows in merged submissions:", submissions.shape[0])

Number of rows in merged submissions: 474


In [4]:
comments = pd.concat([comments1, comments2]).drop_duplicates(["comment_id","redditor_id"]).reset_index(drop=True)
print("Number of rows in merged submissions:", comments.shape[0])
comments['parent_id_clean'] = comments['parent_id'].str.split('_').str[1]

Number of rows in merged submissions: 88792


In [5]:
bot_comments = comments[comments['body'].str.contains("Confirmed: 1 delta awarded", case=False, na=False)]
print("Number of DeltaBot comments awarding delta:", bot_comments.shape[0])
bot_comments.head()

Number of DeltaBot comments awarding delta: 794


,comment_id,link_id,subreddit,parent_id,redditor_id,created_at,body,score,edited,removed,parent_id_clean
1299,jb0aany,11iw30k,changemyview,t1_jb0a96g,ap4fp,2023-03-05T14:17:01+00:00,Confirmed: 1 delta awarded to /u/dollarfrom15c...,{'2024-10-28T21:06:44': 2},False,NaN,jb0a96g
1343,jb157kh,11iw30k,changemyview,t1_jb1554e,ap4fp,2023-03-05T18:19:51+00:00,Confirmed: 1 delta awarded to /u/dollarfrom15c...,{'2024-10-28T21:06:44': 3},False,NaN,jb1554e
1346,jb18kzc,11iw30k,changemyview,t1_jb18ha3,ap4fp,2023-03-05T18:41:58+00:00,Confirmed: 1 delta awarded to /u/Exis007 ([88∆...,{'2024-10-28T21:06:45': 0},False,NaN,jb18ha3
1368,jb16ef7,11iw30k,changemyview,t1_jb16b2o,ap4fp,2023-03-05T18:27:42+00:00,Confirmed: 1 delta awarded to /u/nylockian ([3...,{'2024-10-28T21:06:55': 1},False,NaN,jb16b2o
1373,jb0kxco,11iw30k,changemyview,t1_jb0kvfz,ap4fp,2023-03-05T15:51:38+00:00,Confirmed: 1 delta awarded to /u/nylockian ([1...,{'2024-10-28T21:06:57': 0},False,NaN,jb0kvfz


In [6]:
output_file = "../data/bot_comments.csv"
bot_comments.to_csv(output_file, index=False)
print("Saved file to:", output_file)

Saved file to: ../data/bot_comments.csv


In [7]:
op_delta_comments = comments[comments['comment_id'].isin(bot_comments['parent_id_clean'])]
print("Number of OP comments awarding delta:", op_delta_comments.shape[0])
#opd=op_delta_comments.merge(bot_comments,how="outer",left_on="comment_id",right_on="parent_id_clean")

Number of OP comments awarding delta: 793


In [8]:
d_comments = comments[comments['body'].str.contains("!delta\n\n", case=False, na=False)]
print("Number d:", d_comments.shape[0])
d_comments.tail()

Number d: 157


,comment_id,link_id,subreddit,parent_id,redditor_id,created_at,body,score,edited,removed,parent_id_clean
84087,hi625b1,qgamyj,changemyview,t1_hi4yvfe,3piy84ie,2021-10-26 23:34:12+00,!delta\n\nYou make a good point that excuses w...,"{""2024-11-01T19:49:50"":2}",False,NaN,hi4yvfe
84095,hi67gox,qgamyj,changemyview,t1_hi5nv87,3piy84ie,2021-10-27 00:11:39+00,!delta\n\nYou've shown me that there's a lot m...,"{""2024-11-01T19:49:54"":1}",False,NaN,hi5nv87
86055,ekcmf43,bamauu,changemyview,t1_ekclaz8,i5rnqjm,2019-04-08 01:24:42+00,!delta\n\nThanks for writing back.\n\nAny tips...,"{""2024-11-01T20:07:08"":1}",False,NaN,ekclaz8
87125,dye5u7w,8grirz,changemyview,t1_dye3uzr,ov4fv,2018-05-03 19:43:36+00,Fantastic sources. !delta\n\nWhat do you think...,"{""2024-11-01T20:22:49"":7}",False,NaN,dye3uzr
87285,hjxm12x,qps0qw,changemyview,t1_hjw76pu,9mhwv,2021-11-09 14:32:46+00,"!delta\n\nI will concede that, thanks to your ...","{""2024-11-01T20:24:24"":2}",False,NaN,hjw76pu


In [9]:
output_file = "../data/op_delta_comments.csv"
op_delta_comments.to_csv(output_file, index=False)
print("Saved file to:", output_file)

Saved file to: ../data/op_delta_comments.csv


In [10]:
delta_comments = comments[comments['comment_id'].isin(op_delta_comments['parent_id_clean'])]
delta_comments['parent_id_clean'] = delta_comments['parent_id'].str.split('_').str[1]
print("Number of comments awarded delta:", delta_comments.shape[0])

Number of comments awarded delta: 768


/var/folders/z_/3vpzgjpj4dv90c8dxq0x3ww80000gn/T/ipykernel_96467/1085101165.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delta_comments['parent_id_clean'] = delta_comments['parent_id'].str.split('_').str[1]


In [11]:
output_file = "../data/delta_comments.csv"
delta_comments.to_csv(output_file, index=False)
print("Saved file to:", output_file)

Saved file to: ../data/delta_comments.csv


In [12]:
# Create binary has_delta in comments table
comments['has_delta'] = comments['comment_id'].isin(delta_comments['comment_id']).astype(int)
output_file = "../data/comments_with_has_delta_column.csv"
comments.to_csv(output_file, index=False)
print("Saved file to:", output_file)

Saved file to: ../data/comments_with_has_delta_column.csv


In [13]:
#double check
count_has_delta = comments[comments['has_delta'] == 1].shape[0]
print("Number of comments with has_delta=1:", count_has_delta)

Number of comments with has_delta=1: 768


In [14]:
# Create binary has_delta in submissions table
submissions["has_delta"] = submissions["flair"].apply(lambda x: 1 if json.loads(x)['link'] == "Delta(s) from OP" else 0)
output_file = "../data/submissions_with_has_delta_column.csv"
submissions.to_csv(output_file, index=False)
print("Saved file to:", output_file)

Saved file to: ../data/submissions_with_has_delta_column.csv


In [44]:
submissions_with_deltas = submissions[submissions['has_delta'] == 1]
count_submissions_with_delta = submissions_with_deltas.shape[0]
print("Number of submissions with delta(s) awarded:", count_submissions_with_delta)


Number of submissions with delta(s) awarded: 243


In [49]:
keywords = [
    "climate", "environment", "environmental", "green energy", "global warming", "greenhouse gases", "fossil fuels", 
    "renewable", "net zero", "environmentally", "sustainable", "environmentalism", "carbon", "vegan",
    "footprint", "COP", "IPCC", "plastics", "degradable", "biodegradable",
    "compostable", "greenwashing", "green jobs", "nature-friendly", "overconsumption"
]

pattern = '|'.join(keywords)
submissions_with_deltas = submissions_with_deltas[submissions_with_deltas['text'].str.contains(pattern, case=False, na=False)]
print("Number of filtered submissions:", submissions_with_deltas.shape[0])
submissions_with_deltas.head()

Number of filtered submissions: 236


,submission_id,redditor_id,created_at,title,text,subreddit,permalink,attachment,flair,awards,score,upvote_ratio,num_comments,edited,archived,removed,poll,has_delta
0,16yqq5w,e19ujn874,2023-10-03T14:34:24+00:00,CMV: Conservative apathy towards climate chang...,When you ask the average conservative why they...,changemyview,https://www.reddit.com/r/changemyview/comments...,NaN,"{""link"": ""Delta(s) from OP"", ""author"": ""7∆""}","{""list"": null, ""total_awards_count"": 0, ""total...","{""2024-10-27T12:01:35"": 1318}","{""2024-10-27T12:01:35"": 0.8}","{""2024-10-27T12:01:35"": 991}",True,True,False,NaN,1
2,11iw30k,1a95yjyf,2023-03-05T13:33:28+00:00,CMV: the reason why people aren’t having kids ...,Time and time again I read a new article about...,changemyview,https://www.reddit.com/r/changemyview/comments...,NaN,"{""link"": ""Delta(s) from OP"", ""author"": null}","{""list"": null, ""total_awards_count"": 0, ""total...","{""2024-10-27T12:03:45"": 2108}","{""2024-10-27T12:03:45"": 0.78}","{""2024-10-27T12:03:45"": 632}",False,True,False,NaN,1
3,18qvnpf,7umpg385,2023-12-26T01:55:50+00:00,CMV: Most Americans don’t care enough about cl...,I only say Americans because I am one and don’...,changemyview,https://www.reddit.com/r/changemyview/comments...,NaN,"{""link"": ""Delta(s) from OP"", ""author"": null}","{""list"": null, ""total_awards_count"": 0, ""total...","{""2024-10-27T12:01:55"": 396}","{""2024-10-27T12:01:55"": 0.65}","{""2024-10-27T12:01:55"": 834}",True,True,False,NaN,1
4,1fodkwe,pbcmpax4,2024-09-24T16:16:53+00:00,CMV: Climate change denial or approval should ...,"I""m talking here from an European perspective....",changemyview,https://www.reddit.com/r/changemyview/comments...,NaN,"{""link"": ""Delta(s) from OP"", ""author"": null}","{""list"": null, ""total_awards_count"": 0, ""total...","{""2024-10-27T12:02:04"": 0}","{""2024-10-27T12:02:04"": 0.23}","{""2024-10-27T12:02:04"": 290}",False,False,False,NaN,1
6,14z908l,56eh3pnr,2023-07-14T08:56:08+00:00,CMV: Climate change is real but blocking roads...,I think a lot of people have seen climate chan...,changemyview,https://www.reddit.com/r/changemyview/comments...,NaN,"{""link"": ""Delta(s) from OP"", ""author"": null}","{""list"": null, ""total_awards_count"": 0, ""total...","{""2024-10-27T12:02:14"": 269}","{""2024-10-27T12:02:14"": 0.7}","{""2024-10-27T12:02:14"": 452}",False,True,False,NaN,1


In [50]:
output_file = "../data/submissions_with_deltas.csv"
submissions_with_deltas.to_csv(output_file, index=False)
print("Saved file to:", output_file)

Saved file to: ../data/submissions_with_deltas.csv


In [51]:
comments_from_submissions_with_deltas = comments[comments['link_id'].isin(submissions_with_deltas['submission_id'])]
print("Number of comments associated with submissions where OP awarded delta(s):", comments_from_submissions_with_deltas.shape[0])
comments_from_submissions_with_deltas.head()
output_file = "../data/comments_from_submissions_with_deltas.csv"
comments_from_submissions_with_deltas.to_csv(output_file, index=False)
print("Saved file to:", output_file)

Number of comments associated with submissions where OP awarded delta(s): 48087
Saved file to: ../data/comments_from_submissions_with_deltas.csv


In [52]:
delta_count_in_comments_from_submissions = comments_from_submissions_with_deltas[comments_from_submissions_with_deltas['has_delta'] == 1].shape[0]
print("Number of comments from submissions with deltas that were awarded delta:", delta_count_in_comments_from_submissions)

Number of comments from submissions with deltas that were awarded delta: 562
